In [1]:
from elasticsearch import Elasticsearch, helpers
import os
import re
import logging

In [2]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connected')
    else:
        print('Awww it could not connect!')
    return _es
if __name__ == '__main__':
  logging.basicConfig(level=logging.ERROR)

In [3]:
es = connect_elasticsearch()

Yay Connected


## Load Query File

In [4]:
query_file_path = "D:\\CS 6200\\AP_DATA\\query_desc.51-100.short.txt"
queries = dict()
with open(query_file_path) as f:
    for line in f:
        if len(line.strip()) > 0:
            query_no = re.search(r'\d+', line).group()
            line = line.lstrip('0123456789.- ')
            queries[query_no] = line.rstrip()

In [5]:
doclist_file_path = "D:\\CS 6200\\AP_DATA\\doclist_new_0609.txt"
doclist = []
with open(doclist_file_path) as f:
    for line in f:
        line = line.lstrip('0123456789.- ')
        doclist.append(line.rstrip())

# ES Built In

In [6]:
query_nos_list = list(queries.keys())

In [7]:
def write_to_file(file, query_id, docid, score, rank):
    outF = open(file, "a")
    string = ""
    string = str(query_id) + " Q0 " + str(docid) + "  " + str(rank) + "  "+str(score) + " Exp\n" 
    outF.write(string)
    outF.close()

In [8]:
len_queries_list = len(query_nos_list)

In [9]:
for i in range(0, len_queries_list):
    len_hits = 0
    res = es.search(index="assignment1", body={"query": {"match": {"text": queries[query_nos_list[i]]}}, "size":1000})
    if(res["hits"]["total"]["value"]>1000):
        len_hits = 1000
    else:
        len_hits = res["hits"]["total"]["value"]
    for j in range(0, len_hits):
        write_to_file("D:\\CS 6200\\AP_DATA\\results_esbuiltin.txt", query_nos_list[i], res["hits"]["hits"][j]["_id"], res["hits"]["hits"][j]["_score"], j+1)
